In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [10]:
train_dir = r'path to train dataset'
val_dir = r'path to validation dataset'
test_dir = r'path to test dataset'

# Image Data Generators
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 1506 images belonging to 3 classes.
Found 215 images belonging to 3 classes.
Found 431 images belonging to 3 classes.


In [11]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [12]:
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)  # Assuming 3 classes

model = Model(inputs=base_model.input, outputs=predictions)


In [13]:
for layer in base_model.layers:
    layer.trainable = False


In [14]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [15]:
# Early Stopping Callback
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    verbose=1,
    mode='max',
    restore_best_weights=True
)

In [16]:
# Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=100,  # Set a high number for epochs; early stopping will handle stopping early.
    callbacks=[early_stopping]
)

Epoch 1/100
47/47 [==============================] - 540s 11s/step - loss: 0.6493 - accuracy: 0.7815 - val_loss: 0.2815 - val_accuracy: 0.9115
Epoch 2/100
47/47 [==============================] - 491s 10s/step - loss: 0.2230 - accuracy: 0.9193 - val_loss: 0.3033 - val_accuracy: 0.8802
Epoch 3/100
47/47 [==============================] - 505s 11s/step - loss: 0.1818 - accuracy: 0.9322 - val_loss: 0.2434 - val_accuracy: 0.9323
Epoch 4/100
47/47 [==============================] - 509s 11s/step - loss: 0.1287 - accuracy: 0.9573 - val_loss: 0.1809 - val_accuracy: 0.9531
Epoch 5/100
47/47 [==============================] - 541s 12s/step - loss: 0.0908 - accuracy: 0.9681 - val_loss: 0.1666 - val_accuracy: 0.9531
Epoch 6/100
47/47 [==============================] - 507s 11s/step - loss: 0.0635 - accuracy: 0.9801 - val_loss: 0.1519 - val_accuracy: 0.9479
Epoch 7/100
47/47 [==============================] - 478s 10s/step - loss: 0.0469 - accuracy: 0.9905 - val_loss: 0.1563 - val_accuracy: 0.9583

In [17]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test accuracy: {test_accuracy}')

13/13 [==============================] - 86s 6s/step - loss: 0.1418 - accuracy: 0.9519
Test accuracy: 0.9519230723381042


In [18]:
model.save('vgg16_image_classification_model.h5')

C:\Users\VARUN PN\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
model = tf.keras.models.load_model('vgg16_image_classification_model.h5')

# Preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Load the image
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize the image
    return img_array

# Function to make predictions
def predict_image(img_path):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    class_indices = {v: k for k, v in train_generator.class_indices.items()}  # Get class labels
    import json
    with open("class_names.json","w") as f:
        json.dump(class_indices,f,indent=4)
    predicted_class = np.argmax(predictions, axis=1)
    predicted_label = class_indices[predicted_class[0]]
    return predicted_label, predictions

# Test the model with an image path
img_path = r"Data" # Path to test image
predicted_label, predictions = predict_image(img_path)
print(f'Predicted Label: {predicted_label}')
print(f'Predictions: {predictions}')


1/1 [==============================] - 0s 367ms/step
Predicted Label: Potato___healthy
Predictions: [[2.6475516e-04 1.3603952e-02 9.8613131e-01]]
